# OrientDB tutorial

## Prerequisites

### Documentation

You will find all documentation for :
* [OrientDB SQL reference](http://www.orientdb.com/docs/last/SQL-Functions.html)
* [Orientdb python client](http://orientdb.com/docs/last/PyOrient-Client.html#working-with-the-client)

## Import libraries

In [53]:
import pyorient

In [54]:
ROOT_PASSWORD = "root"
client = pyorient.OrientDB("localhost", 2424)
session_id = client.connect("root", ROOT_PASSWORD)

In [55]:
print(client.db_list())

{{'databases': {}}}


## I. Quick start

### Creating the database

**Q:** Create a database `gods` as a `GRAPH_DATABASE` in `MEMORY_STORAGE_TYPE`. 

We will use it to store relationships between Greek deities.

In [56]:
client.db_create("gods", pyorient.DB_TYPE_GRAPH, pyorient.STORAGE_TYPE_MEMORY)

**Q:** Connect your pyorient client to the `gods` database.

In [57]:
client.db_open("gods", "root", ROOT_PASSWORD)

**Q:** You should now be able to launch OrientDB queries through the Python client with the [command()](http://orientdb.com/docs/last/PyOrient-Client-Command.html) function. 

You should think of OrientDB as a Graph-Document database for the following questions. Each vertex and edge will contain information on it inside a JSON document.

Create a new Vertex with content `{name: 'Zeus', symbol: 'thunder'}`. The [CREATE VERTEX : Create a vertex using JSON content](http://orientdb.com/docs/last/SQL-Create-Vertex.html) doc page should help you.

In [58]:
vertex_content = {
    "name": "Zeus",
    "symbol": "thunder"
}
command = "CREATE VERTEX V CONTENT {}".format(vertex_content)

# Execute the command to create the vertex
result = client.command(command)

# Print the result to confirm creation
print(f"Vertex created: {result}")

Vertex created: [<pyorient.otypes.OrientRecord object at 0x0000028575641640>]


You have created a VERTEX in the previous question. The VERTEX is a [class](https://orientdb.com/docs/last/Tutorial-Classes.html) of OrientDB which defines a record that can be linked to others through EDGE instances.

You can find all VERTEX created in the database with a SQL command on the `V` table, like `SELECT * FROM V`. 

**Q:** Print all current vertices in `gods`, it should only have `Zeus` though for now.

In [59]:
# Execute the SQL command to retrieve all vertices
query = "SELECT * FROM V"
result = client.command(query)

# Print the result, which will contain all the vertices
print("Current vertices in the 'gods' database:")
for vertex in result:
    print(vertex)


Current vertices in the 'gods' database:
{'@V':{'name': 'Zeus', 'symbol': 'thunder'},'version':1,'rid':'#9:0'}


**Q:** Create new vertices with content : 
```
{name:Héra, symbol:tiara}
{name:Poséidon, symbol:trident}
{name:Athena, symbol:helmet}
{name:Arès, symbol:weapons} 
```

In [60]:
# Create new vertices with the given content
vertices_data = [
    {"name": "Héra", "symbol": "tiara"},
    {"name": "Poséidon", "symbol": "trident"},
    {"name": "Athena", "symbol": "helmet"},
    {"name": "Arès", "symbol": "weapons"}
]

# Create each vertex
for data in vertices_data:
    query = f"CREATE VERTEX V SET name = '{data['name']}', symbol = '{data['symbol']}'"
    result = client.command(query)
    print(f"Created vertex: {data['name']}")

Created vertex: Héra
Created vertex: Poséidon
Created vertex: Athena
Created vertex: Arès


**Q:** Display all vertices with name = `Arès`

In [61]:
print(client.command("SELECT * FROM V WHERE name = 'Arès'"))

**Q:** Create an EDGE from `Zeus` to `Poséidon` with the content `{kind: 'sibling'}

In [62]:
result = client.command("CREATE EDGE E FROM (SELECT FROM V WHERE name = 'Zeus') TO (SELECT FROM V WHERE name = 'Poséidon') SET kind = 'sibling'")
print(result)

**Q:** Redisplay all vertices, discuss.

In [63]:
print("Current vertices in the 'gods' database:")
for vertex in client.command("SELECT * FROM V"):
    print(vertex)
    

Current vertices in the 'gods' database:
{'@V':{'name': 'Zeus', 'symbol': 'thunder', 'out_': <pyorient.otypes.OrientBinaryObject object at 0x00000285755F03B0>},'version':2,'rid':'#9:0'}
{'@V':{'name': 'Héra', 'symbol': 'tiara'},'version':1,'rid':'#10:0'}
{'@V':{'name': 'Poséidon', 'symbol': 'trident', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x00000285756D31D0>},'version':2,'rid':'#11:0'}
{'@V':{'name': 'Athena', 'symbol': 'helmet'},'version':1,'rid':'#12:0'}
{'@V':{'name': 'Arès', 'symbol': 'weapons'},'version':1,'rid':'#13:0'}


**Q:** Display all edges. They are contained in the class `E`

In [64]:
print("Current edges in the 'gods' database:")
for edge in client.command("SELECT * FROM E"):
    print(edge)

Current edges in the 'gods' database:
{'@E':{'kind': 'sibling', 'out': <pyorient.otypes.OrientRecordLink object at 0x0000028575634740>, 'in': <pyorient.otypes.OrientRecordLink object at 0x00000285756D2120>},'version':1,'rid':'#17:0'}


Two fields on vertices have appeared, containing the outgoing (out_) and incoming (in_) links.

At the edge level, two fields point to the original (out) and destination (in) vertices.

**Q:** Lets create some more edges :

* Zeus > Héra (sibling)
* Zeus > Arès (father)
* Zeus > Athena (father)
* Héra > Arès (mother)
* Héra > Zeus (sibling)
* Poséidon > Zeus (sibling)

_Hint 1 :_ check [the CREATE EDGE doc page](http://orientdb.com/docs/last/SQL-Create-Edge.html) to find an example for creating edges on vertices using subqueries so you can run queries to fetch the vertices before creating an edge in between.

_Hint 2 :_ after you have found the command to create edges between vertices with sub-queries, you should be well-versed enough in Python to create a list of all edges in the question, and loop the command on each element of the list to create all edges in one go =)

In [65]:
edges_data = [
    {"from": "Zeus", "to": "Héra", "kind": "sibling"},
    {"from": "Zeus", "to": "Arès", "kind": "father"},
    {"from": "Zeus", "to": "Athena", "kind": "father"},
    {"from": "Héra", "to": "Arès", "kind": "mother"},
    {"from": "Héra", "to": "Zeus", "kind": "sibling"},
    {"from": "Poséidon", "to": "Zeus", "kind": "sibling"}
]

for data in edges_data:
    query = f"CREATE EDGE E FROM (SELECT FROM V WHERE name = '{data['from']}') TO (SELECT FROM V WHERE name = '{data['to']}') SET kind = '{data['kind']}'"
    result = client.command(query)
    print(f"Created edge: {data['from']} > {data['to']}")
    
    

Created edge: Zeus > Héra
Created edge: Zeus > Arès


Created edge: Zeus > Athena
Created edge: Héra > Arès
Created edge: Héra > Zeus
Created edge: Poséidon > Zeus


### Looking for data

**Q:** Using [out()](http://orientdb.com/docs/last/Tutorial-Working-with-graphs.html#querying-graphs) function, display all vertices connected and outgoing from Zeus.

You should use the EXPAND() special function to transform the vertex collection in the result-set by expanding it, making the results more readable.

In [66]:
query = "SELECT expand(out()) FROM V WHERE name = 'Zeus'"
result = client.command(query)

print("Vertices connected and outgoing from Zeus:")
for vertex in result:
    print(vertex)

Vertices connected and outgoing from Zeus:
{'@V':{'name': 'Poséidon', 'symbol': 'trident', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x000002857544CBF0>, 'out_': <pyorient.otypes.OrientBinaryObject object at 0x000002857544C890>},'version':3,'rid':'#11:0'}
{'@V':{'out_': <pyorient.otypes.OrientBinaryObject object at 0x00000285756D1730>, 'name': 'Héra', 'symbol': 'tiara', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x00000285756D2600>},'version':4,'rid':'#10:0'}
{'@V':{'in_': <pyorient.otypes.OrientBinaryObject object at 0x00000285756D3860>, 'name': 'Arès', 'symbol': 'weapons'},'version':3,'rid':'#13:0'}
{'@V':{'name': 'Athena', 'symbol': 'helmet', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x00000285756D3D70>},'version':2,'rid':'#12:0'}


**Q:** Display all vertices which got a father (the vertices which are the destination of an arc whose kind attribute is father).

_Hint: You can notice that we use the field `in` the arc, and not the function `in()` which applies to vertices._

In [67]:
result = client.command("SELECT expand(in) FROM E where kind = 'father'")

print("Vertices with a father:")
for vertex in result:
    print(vertex)

Vertices with a father:
{'@V':{'in_': <pyorient.otypes.OrientBinaryObject object at 0x00000285755F2660>, 'name': 'Arès', 'symbol': 'weapons'},'version':3,'rid':'#13:0'}
{'@V':{'name': 'Athena', 'symbol': 'helmet', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x00000285756D1D60>},'version':2,'rid':'#12:0'}


**Q:** As in SQL, the operator `in` used in a clause `where` allows to restrict the possible values with an embedded query _(where ... in (select ...))_. 

Display the mothers, by displaying the vertices where an outgoing arc is part of the arcs where kind is a mother.

In [68]:
print("Vertices with a mother:")
result = client.command("SELECT expand(in) FROM E where kind = 'mother'")
for vertex in result:
    print(vertex)

Vertices with a mother:
{'@V':{'in_': <pyorient.otypes.OrientBinaryObject object at 0x0000028575640CB0>, 'name': 'Arès', 'symbol': 'weapons'},'version':3,'rid':'#13:0'}


**Q:** Display the brothers and sisters of Zeus (the destination summits of an arc whose kind is sibling and whose original summit is Zeus).

In [69]:
#*Q:** Display the brothers and sisters of Zeus (the destination summits of an arc whose kind is sibling and whose original summit is Zeus)
query = "SELECT expand(out()) FROM (SELECT expand(out('sibling')) FROM V WHERE name = 'Zeus')"
result = client.command(query)

print("Brothers and sisters of Zeus:")
for vertex in result:
    print(vertex)

Brothers and sisters of Zeus:


In [70]:
#Display the brothers and sisters of Zeus
query = "SELECT expand(out()) FROM (SELECT expand(out(sibling)) FROM V WHERE name = 'Zeus') WHERE @class = 'V'"
result = client.command(query)

print("Brothers and sisters of Zeus:")
for vertex in result:
    print(vertex)

Brothers and sisters of Zeus:
{'@V':{'out_': <pyorient.otypes.OrientBinaryObject object at 0x00000285756362D0>, 'in_': <pyorient.otypes.OrientBinaryObject object at 0x0000028575634080>, 'name': 'Zeus', 'symbol': 'thunder'},'version':7,'rid':'#9:0'}
{'@V':{'in_': <pyorient.otypes.OrientBinaryObject object at 0x00000285756D2690>, 'name': 'Arès', 'symbol': 'weapons'},'version':3,'rid':'#13:0'}
{'@V':{'out_': <pyorient.otypes.OrientBinaryObject object at 0x00000285756D2EA0>, 'in_': <pyorient.otypes.OrientBinaryObject object at 0x00000285756D0A40>, 'name': 'Zeus', 'symbol': 'thunder'},'version':7,'rid':'#9:0'}


## Modeling a Product Recommendation System

You are currently modeling the data of a product recommendation system with OrientDB.

The main purpose of such a system is to answer the question "which products were purchased by their people who purchased product X? »

Purchased products have only one name field. They are purchased by people who have a nickname.

When a person buys a product, the date of purchase is stored. 

Instead of working with "anonymous" vertices and arcs, you will use classes. The `create class` command allows you to create custom classes.

The vertex classes must extend V, the arc classes must extend E.

**Q:** Create an `eCommerce` database, and the necessary classes to model the system.

PS : you can view all classes in the database with :

```python
for name in client.command("SELECT name FROM (SELECT expand(classes) FROM metadata:schema)"):
    print(name)
```

In [71]:
client.db_create("eCommerce", pyorient.DB_TYPE_GRAPH, pyorient.STORAGE_TYPE_MEMORY)

client.db_open("eCommerce", "root", ROOT_PASSWORD)

client.command("CREATE CLASS Product EXTENDS V")
client.command("CREATE CLASS Person EXTENDS V")
client.command("CREATE CLASS Purchase EXTENDS E")

for name in client.command("SELECT name FROM (SELECT expand(classes) FROM metadata:schema)"):
   print(name)

{{'name': 'Purchase'},'version':0,'rid':'#-2:0'}
{{'name': 'Person'},'version':0,'rid':'#-2:1'}
{{'name': 'ORestricted'},'version':0,'rid':'#-2:2'}
{{'name': 'OTriggered'},'version':0,'rid':'#-2:3'}
{{'name': 'OIdentity'},'version':0,'rid':'#-2:4'}
{{'name': 'OFunction'},'version':0,'rid':'#-2:5'}
{{'name': 'V'},'version':0,'rid':'#-2:6'}
{{'name': 'OSchedule'},'version':0,'rid':'#-2:7'}
{{'name': 'E'},'version':0,'rid':'#-2:8'}
{{'name': 'OSequence'},'version':0,'rid':'#-2:9'}
{{'name': 'OUser'},'version':0,'rid':'#-2:10'}
{{'name': 'ORole'},'version':0,'rid':'#-2:11'}
{{'name': 'Product'},'version':0,'rid':'#-2:12'}


**Q:** Create the following products: `spaghetti`, `bolognese sauce`, `cheese`, `apple`.

In [72]:
# Create the products
products = ["spaghetti", "bolognese sauce", "cheese", "apple"]
for product in products:
    query = f"CREATE VERTEX Product SET name = '{product}'"
    result = client.command(query)
    print(f"Created product: {product}")

Created product: spaghetti
Created product: bolognese sauce
Created product: cheese
Created product: apple


**Q:** Create the following people: `peter`, `meredith`.

In [73]:
# Create the people
people = ["peter", "meredith"]
for person in people:
    query = f"CREATE VERTEX Person SET nickname = '{person}'"
    result = client.command(query)
    print(f"Created person: {person}")

Created person: peter
Created person: meredith


**Q:** Create the following purchases: 
- peter > spaghetti + cheese on 20/01/2016 
- meredith > cheese + apple + bolognese sauce on 22/01/2016
- peter > spaghetti + bolognese sauce on 27/01/2016


In [74]:
# Create the purchases
purchases = [
    {"person": "peter", "products": ["spaghetti", "cheese"], "date": "20/01/2016"},
    {"person": "meredith", "products": ["cheese", "apple", "bolognese sauce"], "date": "22/01/2016"},
    {"person": "peter", "products": ["spaghetti", "bolognese sauce"], "date": "27/01/2016"}
]

for purchase in purchases:
    person = purchase["person"]
    products = purchase["products"]
    date = purchase["date"]
    for product in products:
        query = f"CREATE EDGE Purchase FROM (SELECT FROM Person WHERE nickname = '{person}') TO (SELECT FROM Product WHERE name = '{product}') SET date = '{date}'"
        result = client.command(query)
        print(f"Created purchase: {person} > {product} on {date}")

Created purchase: peter > spaghetti on 20/01/2016
Created purchase: peter > cheese on 20/01/2016
Created purchase: meredith > cheese on 22/01/2016
Created purchase: meredith > apple on 22/01/2016


Created purchase: meredith > bolognese sauce on 22/01/2016
Created purchase: peter > spaghetti on 27/01/2016
Created purchase: peter > bolognese sauce on 27/01/2016


In [75]:
# Display the purchases
query = "SELECT expand(out()) FROM Person"
result = client.command(query)

print("Purchases:")
for person in result:
    print(person)

Purchases:
{'@Product':{'in_Purchase': <pyorient.otypes.OrientBinaryObject object at 0x0000028575634080>, 'name': 'spaghetti'},'version':3,'rid':'#25:0'}
{'@Product':{'in_Purchase': <pyorient.otypes.OrientBinaryObject object at 0x00000285756D3A10>, 'name': 'cheese'},'version':3,'rid':'#27:0'}
{'@Product':{'in_Purchase': <pyorient.otypes.OrientBinaryObject object at 0x0000028575641400>, 'name': 'spaghetti'},'version':3,'rid':'#25:0'}
{'@Product':{'in_Purchase': <pyorient.otypes.OrientBinaryObject object at 0x0000028575641490>, 'name': 'bolognese sauce'},'version':3,'rid':'#26:0'}
{'@Product':{'in_Purchase': <pyorient.otypes.OrientBinaryObject object at 0x0000028575643620>, 'name': 'cheese'},'version':3,'rid':'#27:0'}
{'@Product':{'name': 'apple', 'in_Purchase': <pyorient.otypes.OrientBinaryObject object at 0x0000028575647E60>},'version':2,'rid':'#28:0'}
{'@Product':{'in_Purchase': <pyorient.otypes.OrientBinaryObject object at 0x0000028575647F80>, 'name': 'bolognese sauce'},'version':3,'

**Q:** Who bought Bolognese sauce?

In [76]:
query = "SELECT expand(in()) FROM Product WHERE name = 'bolognese sauce'"
result = client.command(query)

print("People who bought Bolognese sauce:")
for person in result:
    print(person)

People who bought Bolognese sauce:
{'@Person':{'out_Purchase': <pyorient.otypes.OrientBinaryObject object at 0x00000285756D1340>, 'nickname': 'meredith'},'version':4,'rid':'#34:0'}
{'@Person':{'out_Purchase': <pyorient.otypes.OrientBinaryObject object at 0x0000028575647DD0>, 'nickname': 'peter'},'version':5,'rid':'#33:0'}


**Q:** It is possible to link the `out` and `in` navigation functions. What products are purchased with Bolognese sauce? 

In [77]:
query = "SELECT expand(in().out()) FROM Product WHERE name = 'bolognese sauce'"
result = client.command(query)

print("Products purchased with Bolognese sauce:")
for product in result:
    print(product)

Products purchased with Bolognese sauce:
{'@Product':{'in_Purchase': <pyorient.otypes.OrientBinaryObject object at 0x0000028575643050>, 'name': 'cheese'},'version':3,'rid':'#27:0'}
{'@Product':{'name': 'apple', 'in_Purchase': <pyorient.otypes.OrientBinaryObject object at 0x00000285756D1C10>},'version':2,'rid':'#28:0'}
{'@Product':{'in_Purchase': <pyorient.otypes.OrientBinaryObject object at 0x00000285756D38C0>, 'name': 'bolognese sauce'},'version':3,'rid':'#26:0'}
{'@Product':{'in_Purchase': <pyorient.otypes.OrientBinaryObject object at 0x00000285756D2D50>, 'name': 'spaghetti'},'version':3,'rid':'#25:0'}
{'@Product':{'in_Purchase': <pyorient.otypes.OrientBinaryObject object at 0x00000285756D3A10>, 'name': 'cheese'},'version':3,'rid':'#27:0'}
{'@Product':{'in_Purchase': <pyorient.otypes.OrientBinaryObject object at 0x00000285756474A0>, 'name': 'spaghetti'},'version':3,'rid':'#25:0'}
{'@Product':{'in_Purchase': <pyorient.otypes.OrientBinaryObject object at 0x0000028575646DB0>, 'name': 'b

## Postquisites

Since we create databases in memory, they get destroyed on server shutdown.